In [1]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Basic word2vec example."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf



In [2]:
# Step 1: 读取文件
url = './QuanSongCi.txt'

# Read the data into a list of strings.
def read_data(url):
    
  with open(url,'r',encoding='UTF-8') as f:
    data = f.read().strip('\n')
    result=[]
    result = list(data)
  return result

vocabulary = read_data(url)
print(vocabulary[:100])
print('Data size', len(vocabulary))
type(vocabulary)

# filename = "./QuanSongCi.zip"


# # Read the data into a list of strings.
# def read_data(filename):
#   """Extract the first file enclosed in a zip file as a list of words."""
#   with zipfile.ZipFile(filename) as f:
#     data = tf.compat.as_str(f.read(f.namelist()[0])).split()
#   return data

# vocabulary = read_data(filename)
# print(vocabulary[:100])
# print('Data size', len(vocabulary))



['潘', '阆', '\n', '\n', '酒', '泉', '子', '（', '十', '之', '一', '）', '\n', '\n', '长', '忆', '钱', '塘', '，', '不', '是', '人', '寰', '是', '天', '上', '。', '万', '家', '掩', '映', '翠', '微', '间', '。', '处', '处', '水', '潺', '潺', '。', '\n', '\n', '异', '花', '四', '季', '当', '窗', '放', '。', '出', '入', '分', '明', '在', '屏', '障', '。', '别', '来', '隋', '柳', '几', '经', '秋', '。', '何', '日', '得', '重', '游', '。', '\n', '\n', '酒', '泉', '子', '（', '十', '之', '二', '）', '\n', '\n', '长', '忆', '钱', '塘', '，', '临', '水', '傍', '山', '三', '百', '寺', '。', '僧', '房']
Data size 1903071


list

In [3]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 5000


def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    index = dictionary.get(word, 0)
    if index == 0:  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

# Filling 4 global variables:
# data - list of codes (integers from 0 to vocabulary_size-1).
#   This is the original text but words are replaced by their codes
# count - map of words(strings) to count of occurrences
# dictionary - map of words(strings) to their codes(integers)
# reverse_dictionary - maps codes(integers) to words(strings)
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                            vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0



Most common words (+UNK) [['UNK', 1196], ('。', 149620), ('\n', 117068), ('，', 108451), ('、', 19612)]
Sample data [1503, 1828, 2, 2, 40, 613, 47, 9, 111, 117] ['潘', '阆', '\n', '\n', '酒', '泉', '子', '（', '十', '之']


In [4]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  if data_index + span > len(data):
    data_index = 0
  buffer.extend(data[data_index:data_index + span])
  data_index += span
  for i in range(batch_size // num_skips):
    context_words = [w for w in range(span) if w != skip_window]
    words_to_use = random.sample(context_words, num_skips)
    for j, context_word in enumerate(words_to_use):
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[context_word]
    if data_index == len(data):
      print("到这里啦啦啦啦啦啦啦啦啦",data_index)
      buffer.extend(data[:span])
      data_index = span
    else:
      buffer.append(data[data_index])
      data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])




1828 阆 -> 1503 潘
1828 阆 -> 2 

2 
 -> 1828 阆
2 
 -> 2 

2 
 -> 40 酒
2 
 -> 2 

40 酒 -> 2 

40 酒 -> 613 泉


In [5]:
#保存字典用于RNN
import json
with open("reversed_dictionary.json",'w',encoding='utf-8') as json_file:
    json.dump(reverse_dictionary,json_file,ensure_ascii=False)


In [6]:
#保存字典用于RNN
with open("dictionary.json",'w',encoding='utf-8') as json_file:
    json.dump(dictionary,json_file,ensure_ascii=False)

In [7]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.
num_sampled = 64      # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)


graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  # Explanation of the meaning of NCE loss:
  #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()



In [ ]:
# Step 5: Begin training.
num_steps = 400001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()



Initialized
Average loss at step  0 :  224.681304932
Nearest to 长: 远, 袝, 诙, 姤, 肤, 肝, 乔, 狄,
Nearest to 雨: バ, 褐, 荐, 黔, 庵, 袴, 性, 冀,
Nearest to 天: 嘴, 珉, 孰, 众, 踞, 盱, 眺, 徜,
Nearest to 秋: 泪, 砑, 官, 董, 9, 蔫, 慢, 窍,
Nearest to 处: 豺, 缓, 鄱, 泪, 邪, 聆, 陶, 换,
Nearest to 如: 略, 崖, 敌, 黑, h, 谌, 腕, 歆,
Nearest to 花: 请, 豫, 宇, 萼, 赁, 株, 浓, 遑,
Nearest to 明: 绎, 彼, 咽, 庠, 皤, 息, 邪, 鹍,
Nearest to 山: 回, 煮, 雷, 楹, 穆, 挨, 笠, 榄,
Nearest to 时: 简, 纹, 伏, 戛, 柰, 豪, 靠, 棹,
Nearest to 楼: 绸, 牍, 浦, 劳, 灶, 痴, 稳, 鞘,
Nearest to UNK: 医, 恳, 飒, 彦, 桡, 涵, 豁, 聒,
Nearest to 南: 哨, 景, 功, 悟, 遂, 躲, 幄, 众,
Nearest to 子: 础, 骤, 堵, 裘, 冒, 彳, 捶, 童,
Nearest to 金: 晚, 元, 葵, 胃, 擢, 箴, 仟, 虢,
Nearest to 春: 鹿, 筝, 族, 罗, 笑, 盎, 鄮, 赑,
Average loss at step  2000 :  21.4679469103
Average loss at step  4000 :  5.27971145093
Average loss at step  6000 :  4.88642179573
Average loss at step  8000 :  4.69321194553
Average loss at step  10000 :  4.5923746044
Nearest to 长: 远, 肤, 桐, 卓, 姤, 乔, 写, 勤,
Nearest to 雨: 褐, 荐,  , 棋, UNK, 性, 万, 黔,
Nearest to 天: 众, 化, 和, 眺, 盱, 蔷, 匪, 云,
N

Average loss at step  92000 :  4.05031161332
Average loss at step  94000 :  4.08411049616
Average loss at step  96000 :  4.05318475103
Average loss at step  98000 :  4.07924844944
Average loss at step  100000 :  4.06149120873
Nearest to 长: 鹉, 正, 犹, 展, 酤, 横, 桐, 卓,
Nearest to 雨: 柳, 褐, 雾, 霭, 黔, UNK, 腋, 押,
Nearest to 天: 云, 匪, 倡, 盱, 喜, 眺, 欲, 自,
Nearest to 秋: 春, 溢, 涩, 岩, 湜, 月, 寒, 雪,
Nearest to 处: 优, 聆, 时, 又, 意, 限, 鉏, 粝,
Nearest to 如: 似, h, 在, 伪, 罨, 。, 渎, 涡,
Nearest to 花: 柳, 梅, 叶, 芳, 帽, 竟, 絮, 舻,
Nearest to 明: 十, 绎, 看, 照, 创, 彼, 庠, 朦,
Nearest to 山: 峰, 楹, 树, 穆, 阳, 璈, 皑, 曦,
Nearest to 时: 靠, 年, 处, 了, 第, 饶, 初, 伏,
Nearest to 楼: 阁, 林, 儋, 藜, 台, 曷, 穆, 瞻,
Nearest to UNK: 医, ＿, 铁, 绽, 患, 雨, 阿, 飒,
Nearest to 南: 西, 北, 东, 头, 恝, 楚, 绕, 湖,
Nearest to 子: 寅, 捶, 姜, 恳, 责, 童, 补, 卜,
Nearest to 金: 隅, 盘, 唾, 芭, 嫩, 典, 秉, 元,
Nearest to 春: 秋, 晴, 涩, 懒, 花, 梅, 寡, 柳,
Average loss at step  102000 :  4.08047379625
Average loss at step  104000 :  4.09698136747
Average loss at step  106000 :  4.14657792318
Average loss at step  10

Average loss at step  192000 :  4.00367802012
Average loss at step  194000 :  4.02656137061
Average loss at step  196000 :  4.07432550299
Average loss at step  198000 :  4.06767665946
Average loss at step  200000 :  4.08477843976
Nearest to 长: 鹉, 正, 犹, 酤, 祲, 当, 渌, 平,
Nearest to 雨: 饮, 柳, 押, 霭, 麦, 腋, 蜜, UNK,
Nearest to 天: 匪, 盱, 钩, 欲, 空, 云, 自, 其,
Nearest to 秋: 春, 溢, 寒, 岩, 湜, 雪, 涩, 晚,
Nearest to 处: 优, 聆, 便, 限, 粝, 意, 苦, 又,
Nearest to 如: 似, 渎, h, 伪, 在, 违, 罨, 店,
Nearest to 花: 芳, 柳, 棠, 絮, 、, 梅, 叶, 早,
Nearest to 明: 十, 看, 照, 朦, 今, 绎, 沚, 澄,
Nearest to 山: 峰, 楹, 曦, 逝, 趓, 水, 璈, 祠,
Nearest to 时: 年, 番, 靠, 第, 曾, 汝, 余, 想,
Nearest to 楼: 阁, 瞻, 藜, 林, 台, 城, 儋, 穆,
Nearest to UNK: ＿, 医, 绽, 缱, 藉, 铁, 患, 旆,
Nearest to 南: 西, 北, 东, 楚, 上, 头, 湖, 艹,
Nearest to 子: 寅, 捶, 盱, 恳, 倪, 责, 降, 辈,
Nearest to 金: 唾, 芭, 砂, 嫩, 铅, 秉, 琼, 玉,
Nearest to 春: 秋, 人, 迟, 鲂, 晴, 第, 寡, 芊,
Average loss at step  202000 :  4.05933639336
Average loss at step  204000 :  4.03581329882
Average loss at step  206000 :  4.08564823687
Average loss at step

Average loss at step  292000 :  3.99381699872
Average loss at step  294000 :  4.0174521873
Average loss at step  296000 :  4.02557960677
到这里啦啦啦啦啦啦啦啦啦 1903071
Average loss at step  298000 :  3.97928761029
Average loss at step  300000 :  3.92318539727
Nearest to 长: 宜, 鹉, 正, 当, 常, 祲, 币, 酤,
Nearest to 雨: 腋, 霭, 麦, 饮, 潇, 雪, 雁, 露,
Nearest to 天: 云, 匪, 默, 庑, 空, 盱, 其, 抵,
Nearest to 秋: 春, 寒, 晚, 溢, 湜, 涩, 岩, 涵,
Nearest to 处: 聆, 限, 优, 意, 便, 妨, 粝, 最,
Nearest to 如: 似, 坻, 在, 徂, 伪, 渎, 疑, 成,
Nearest to 花: 芳, 柳, 菊, 红, 纫, 、, 早, 叶,
Nearest to 明: 十, 照, 晴, 光, 澄, 朦, 郴, 沚,
Nearest to 山: 峰, 逝, 楹, 望, 嶂, 水, 峦, 曦,
Nearest to 时: 番, 年, 靠, 第, 霎, 汝, 处, 还,
Nearest to 楼: 城, 阁, 瞻, 藜, 林, 狭, 堂, 儋,
Nearest to UNK: ＿, 绽, 医, 藉, 缱, 录, 粥, 嘻,
Nearest to 南: 西, 北, 东, 湖, 楚, 故, 茝, 头,
Nearest to 子: 倪, 盱, 恳, 寅, 降, 姜, 捶, 匕,
Nearest to 金: 芭, 玉, 琼, 璃, 嫩, 砂, 唾, 瑶,
Nearest to 春: 秋, 冬, 迟, 晴, 鲂, 花, 蚊, 芜,
Average loss at step  302000 :  3.95851428139
Average loss at step  304000 :  3.93499167478
Average loss at step  306000 :  3.94050582838
Av

Average loss at step  392000 :  3.96080070221
Average loss at step  394000 :  3.89444140434
Average loss at step  396000 :  3.92338559318
Average loss at step  398000 :  3.93313621113
Average loss at step  400000 :  3.92919304061
Nearest to 长: 宜, 祲, 常, 当, 鹉, 币, 苦, 正,
Nearest to 雨: 押, 麦, 霭, 露, 腋, 潇, 暴, 蜜,
Nearest to 天: 匪, 盱, 其, 空, 茉, 庑, 鼙, 抵,
Nearest to 秋: 春, 寒, 晚, 涵, 湜, 岁, 挈, 冬,
Nearest to 处: 限, 最, 聆, 优, 便, 妨, 尤, 意,
Nearest to 如: 似, 渎, 在, 伪, h, 隼, 坻, 於,
Nearest to 花: 芳, 菊, 柳, 红, 早, 酡, 棠, 梅,
Nearest to 明: 十, 光, 照, 朦, 看, 沚, 今, 圣,
Nearest to 山: 峰, 逝, 峡, 峦, 水, 楹, 趓, 闳,
Nearest to 时: 番, 靠, 霎, 年, 第, 处, 曾, 场,
Nearest to 楼: 城, 瞻, 阁, 狭, 殿, 堂, 台, 裔,
Nearest to UNK: ＿, 绽, 医, 藉, 缱, 嘻, 艾, 录,
Nearest to 南: 西, 北, 楚, 东, 湖, 头, コ, 迢,
Nearest to 子: 盱, 恳, 责, 倪, 卜, 葛, 辈, 童,
Nearest to 金: 琼, 芭, 璃, 嫩, 瑶, 蘩, 潋, 笙,
Nearest to 春: 秋, 冬, 迟, 鲂, 芊, 晴, 花, 腊,


In [ ]:
# Step 6: Visualize the embeddings.


# pylint: disable=missing-docstring
# Function to draw visualization of distance between embeddings.
def plot_with_labels(low_dim_embs, labels, filename):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
  print(filename)  
  plt.savefig(filename)

try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt
  import matplotlib 
  from pylab import mpl
  
  mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
  mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels, 'tsne.png')

except ImportError as ex:
  print('Please install sklearn, matplotlib, and scipy to show embeddings.')
  print(ex)
    
np.save('embedding.npy', final_embeddings)
print(final_embeddings)

In [ ]:
print(matplotlib.path)